In [1]:
import torch
from tqdm import tqdm
import time
from contextlib import nullcontext
import random
import numpy as np
import os
import math

In [2]:
sentences = [] 
with open("output_data_50.txt","r") as f:
    sentences = f.readlines()

In [3]:
sentences[:10]

['paulina:true, too true, my lord:if, one by one_eno yb eno ,fi:drol ym ,eurt oot ,eurt:aniluap^\n',
 'r he make his wayunder the colour of his usual ga_ag lausu sih fo ruoloc eht rednuyaw sih ekam eh r^\n',
 'sages, and in the end,having my freedom, boast of_fo tsaob ,modeerf ym gnivah,dne eht ni dna ,segas^\n',
 'ng henry virichard:now, clifford, i have single_elgnis evah i ,droffilc ,won:drahciriv yrneh gn^\n',
 "-damn him. be she honour-flaw'd,i have three daug_guad eerht evah i,d'walf-ruonoh ehs eb .mih nmad-^\n",
 " clifford; and that's richard duke of york.king _ gnik.kroy fo ekud drahcir s'taht dna ;droffilc ^\n",
 'and with your queen. i am his cupbearer:if from _ morf fi:reraebpuc sih ma i .neeuq ruoy htiw dna^\n',
 'le:northumberland comes back from bolingbroke.k_k.ekorbgnilob morf kcab semoc dnalrebmuhtron:el^\n',
 ' so unlike yourself?petruchio:tedious it were t_t erew ti suoidet:oihcurtep?flesruoy ekilnu os ^\n',
 'y, proud queen, to make thee blush.to tell thee w_w eeht 

In [4]:
sentences = [] 
with open("output_data_50.txt","r") as f:
    sentences = f.readlines()

text = "".join(sentences[:50000])

In [5]:
# at beginning of the script
device_type = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device_type)
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScale

cuda:0


In [6]:
total_vocab = sorted(list(set("".join(sentences))))
vocab_size = len(total_vocab)

In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(total_vocab) }
itos = { i:ch for i,ch in enumerate(total_vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there^"))
print(decode(encode("hii there")))

[22, 23, 23, 1, 34, 22, 19, 32, 19, 13]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = []
for line in tqdm(sentences[:10]): 
    data.append(torch.tensor(encode(line), dtype=torch.long))
print(data[:5])

100%|██████████| 10/10 [00:00<00:00, 20470.00it/s]

[tensor([30, 15, 35, 26, 23, 28, 15, 10, 34, 32, 35, 19,  6,  1, 34, 29, 29,  1,
        34, 32, 35, 19,  6,  1, 27, 39,  1, 26, 29, 32, 18, 10, 23, 20,  6,  1,
        29, 28, 19,  1, 16, 39,  1, 29, 28, 19, 14, 19, 28, 29,  1, 39, 16,  1,
        19, 28, 29,  1,  6, 20, 23, 10, 18, 32, 29, 26,  1, 39, 27,  1,  6, 19,
        35, 32, 34,  1, 29, 29, 34,  1,  6, 19, 35, 32, 34, 10, 15, 28, 23, 26,
        35, 15, 30, 13,  0]), tensor([32,  1, 22, 19,  1, 27, 15, 25, 19,  1, 22, 23, 33,  1, 37, 15, 39, 35,
        28, 18, 19, 32,  1, 34, 22, 19,  1, 17, 29, 26, 29, 35, 32,  1, 29, 20,
         1, 22, 23, 33,  1, 35, 33, 35, 15, 26,  1, 21, 15, 14, 15, 21,  1, 26,
        15, 35, 33, 35,  1, 33, 23, 22,  1, 20, 29,  1, 32, 35, 29, 26, 29, 17,
         1, 19, 22, 34,  1, 32, 19, 18, 28, 35, 39, 15, 37,  1, 33, 23, 22,  1,
        19, 25, 15, 27,  1, 19, 22,  1, 32, 13,  0]), tensor([33, 15, 21, 19, 33,  6,  1, 15, 28, 18,  1, 23, 28,  1, 34, 22, 19,  1,
        19, 28, 18,  6, 22, 15, 36,

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 20

x = train_data[1][:block_size]
y = train_data[1][1:block_size+1]
print(x, y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

tensor([32,  1, 22, 19,  1, 27, 15, 25, 19,  1, 22, 23, 33,  1, 37, 15, 39, 35,
        28, 18]) tensor([ 1, 22, 19,  1, 27, 15, 25, 19,  1, 22, 23, 33,  1, 37, 15, 39, 35, 28,
        18, 19])
when input is tensor([32]) the target: 1
when input is tensor([32,  1]) the target: 22
when input is tensor([32,  1, 22]) the target: 19
when input is tensor([32,  1, 22, 19]) the target: 1
when input is tensor([32,  1, 22, 19,  1]) the target: 27
when input is tensor([32,  1, 22, 19,  1, 27]) the target: 15
when input is tensor([32,  1, 22, 19,  1, 27, 15]) the target: 25
when input is tensor([32,  1, 22, 19,  1, 27, 15, 25]) the target: 19
when input is tensor([32,  1, 22, 19,  1, 27, 15, 25, 19]) the target: 1
when input is tensor([32,  1, 22, 19,  1, 27, 15, 25, 19,  1]) the target: 22
when input is tensor([32,  1, 22, 19,  1, 27, 15, 25, 19,  1, 22]) the target: 23
when input is tensor([32,  1, 22, 19,  1, 27, 15, 25, 19,  1, 22, 23]) the target: 33
when input is tensor([32,  1, 22, 19,  1,

In [11]:
torch.randint(10, (32,))

tensor([3, 7, 4, 0, 0, 5, 4, 0, 0, 9, 8, 7, 2, 0, 0, 2, 6, 1, 3, 7, 9, 3, 4, 4,
        8, 8, 5, 9, 5, 0, 7, 6])

In [12]:
torch.manual_seed(42)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 16 # what is the maximum context length for predictions?

# poor man's data loader
def get_batch(sample):
    data = sample
    ix = torch.randint(len(data) - block_size + 1, (batch_size,))
    x_temps = [data[i:i+block_size] for i in ix]
    x = []
    for _x in x_temps:
        if len(_x) < block_size:
            while len(_x) < block_size:
                new_element = torch.tensor(encode("^"))
                _x = torch.cat((_x,new_element),0)
        x.append(_x)
    
    y_temps = [data[i+1:i+1+block_size] for i in ix]
    y = []
    for _y in y_temps:
        
        if len(_y) < block_size:
            while len(_y) < block_size:
                new_element = torch.tensor(encode("^"))
                _y = torch.cat((_y,new_element),0)
        y.append(_y)
    
    x = torch.stack(x)
    y = torch.stack(y)
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device_type, non_blocking=True), y.pin_memory().to(device_type, non_blocking=True)
    else:
        x, y = x.to(device_type), y.to(device_type)
    return x, y

In [13]:
train_data[0]

tensor([30, 15, 35, 26, 23, 28, 15, 10, 34, 32, 35, 19,  6,  1, 34, 29, 29,  1,
        34, 32, 35, 19,  6,  1, 27, 39,  1, 26, 29, 32, 18, 10, 23, 20,  6,  1,
        29, 28, 19,  1, 16, 39,  1, 29, 28, 19, 14, 19, 28, 29,  1, 39, 16,  1,
        19, 28, 29,  1,  6, 20, 23, 10, 18, 32, 29, 26,  1, 39, 27,  1,  6, 19,
        35, 32, 34,  1, 29, 29, 34,  1,  6, 19, 35, 32, 34, 10, 15, 28, 23, 26,
        35, 15, 30, 13,  0])

In [14]:
get_batch(train_data[0])

(tensor([[ 6,  1, 27, 39,  1, 26, 29, 32, 18, 10, 23, 20,  6,  1, 29, 28],
         [39, 27,  1,  6, 19, 35, 32, 34,  1, 29, 29, 34,  1,  6, 19, 35],
         [29, 29, 34,  1,  6, 19, 35, 32, 34, 10, 15, 28, 23, 26, 35, 15],
         [34, 29, 29,  1, 34, 32, 35, 19,  6,  1, 27, 39,  1, 26, 29, 32],
         [ 1, 26, 29, 32, 18, 10, 23, 20,  6,  1, 29, 28, 19,  1, 16, 39],
         [28, 29,  1,  6, 20, 23, 10, 18, 32, 29, 26,  1, 39, 27,  1,  6],
         [23, 10, 18, 32, 29, 26,  1, 39, 27,  1,  6, 19, 35, 32, 34,  1],
         [23, 28, 15, 10, 34, 32, 35, 19,  6,  1, 34, 29, 29,  1, 34, 32],
         [ 6, 19, 35, 32, 34,  1, 29, 29, 34,  1,  6, 19, 35, 32, 34, 10],
         [32, 34,  1, 29, 29, 34,  1,  6, 19, 35, 32, 34, 10, 15, 28, 23],
         [34, 32, 35, 19,  6,  1, 27, 39,  1, 26, 29, 32, 18, 10, 23, 20],
         [19, 28, 29,  1,  6, 20, 23, 10, 18, 32, 29, 26,  1, 39, 27,  1],
         [35, 19,  6,  1, 34, 29, 29,  1, 34, 32, 35, 19,  6,  1, 27, 39],
         [34,  1, 29, 29,

In [31]:
config_file = "train_config.py"
with open(config_file) as f:
    print(f.read())
exec(open(config_file).read())

config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str)) and k.find("_") > 1]
config = {k: globals()[k] for k in config_keys} # will be useful for logging

# config for training GPT-2 (124M) down to very nice loss of ~2.85 on 1 node of 8X A100 40GB
# launch as the following (e.g. in a screen session) and wait ~5 days:
# $ torchrun --standalone --nproc_per_node=8 train.py config/train_gpt2.py

wandb_log = True
wandb_project = 'gpt_train_string_reversal'
# wandb_run_name=''

# these make the total batch size be ~0.5M
# 12 batch size * 1024 block size * 5 gradaccum * 8 GPUs = 491,520
batch_size = 16
block_size = 64
gradient_accumulation_steps = 5 * 8


# this makes total number of tokens be 300B
max_iters = 20000
lr_decay_iters = 15000

# eval stuff
eval_interval = 1000
eval_iters = 200
log_interval = 500

# weight decay
weight_decay = 1e-1

# Model stuff
n_embd = 512
n_head = 4
n_layer = 4

learning_rate = 1e-6 # max learning rate
min_lr = 1e-7 # learning_rate / 10 usually


beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
decay_lr = True # whether to decay the learning rate
warmup_iters = 1000 #

In [32]:
config

{'vocab_size': 41,
 'block_size': 64,
 'batch_size': 16,
 'config_file': 'train_config.py',
 'wandb_log': True,
 'wandb_project': 'gpt_train_string_reversal',
 'gradient_accumulation_steps': 40,
 'max_iters': 20000,
 'lr_decay_iters': 20000,
 'eval_interval': 1000,
 'eval_iters': 200,
 'log_interval': 500,
 'weight_decay': 0.1,
 'learning_rate': 1e-06,
 'min_lr': 1e-07,
 'grad_clip': 1.0,
 'decay_lr': True,
 'warmup_iters': 1000,
 'out_dir': './checkpoints/',
 'iter_num': 1000,
 'init_from': 'scratch',
 'local_iter_num': 500,
 'running_mfu': -1.0,
 'sample_cursor': 20001,
 'micro_step': 39}

In [33]:
# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
model_args['vocab_size'] = vocab_size  

In [34]:
from gpt.model import GPTConfig
from gpt.model import GPT

In [35]:
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)

number of parameters: 12.61M


In [36]:
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device_type)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(41, 512)
    (wpe): Embedding(64, 512)
    (drop): Dropout(p=0.4, inplace=False)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=False)
          (c_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.4, inplace=False)
          (resid_dropout): Dropout(p=0.4, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.4, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=512, out_features=41, bias=False)
)

In [37]:
# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
init_from = 'scratch'
# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
# if init_from == 'resume':
#     optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

num decayed parameter tensors: 18, with 12,636,672 parameters
num non-decayed parameter tensors: 9, with 4,608 parameters
using fused AdamW: False


In [38]:
# helps estimate an arbitrarily accurate loss over either split using many batches
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch_classic(split):
    # generate a small batch of data of inputs x and targets y
    data = split
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device_type), y.to(device_type)
    return x , y
    
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    data_dict = {"train" : train_data , "val" : val_data}
    for key in data_dict:
        data = data_dict[key]
        losses = torch.zeros(eval_iters)
        start = random.choice(np.arange(len(data)-eval_iters-1))
        i = 0
        for k in range(start, start + eval_iters):
            # X, Y = get_batch(data[k])
            X, Y = get_batch_classic(data)
            logits, loss = model(X, Y)
            losses[i] = loss.item()
            i+=1
        out[key] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log:
    import wandb
    wandb.init(project=wandb_project, config=config)


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval_loss,█▁
iter,▁
lr,▁
mfu,▁▁▁
time,█▁
train/loss,▁
val/loss,▁
eval_loss,3.38838
iter,0
lr,0.0
mfu,-100.0


In [42]:
# training loop
# X, Y = get_batch(train_data[0]) # fetch the very first batch
X, Y = get_batch_classic(train_data) # fetch the very first batch
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model # unwrap DDP container if needed
running_mfu = -1.0
sample_cursor = 1
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": losses['train'],
                "val/loss": losses['val'],
                "lr": lr,
                "mfu": running_mfu*100, # convert to percentage
            })
        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        logits, loss = model(X, Y)
        loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        # X, Y = get_batch(train_data[sample_cursor])
        get_batch_classic(train_data)
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
        sample_cursor+=1
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
        wandb.log({"eval_loss": lossf, "time": dt*1000 , "mfu": running_mfu*100})
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

iter 500: loss 3.1984, time 278.98ms, mfu -100.00%
step 1000: train loss 3.1706, val loss 3.1660
saving checkpoint to ./checkpoints/
iter 1000: loss 2.6102, time 11330.25ms, mfu 0.09%
iter 1500: loss 2.1281, time 281.28ms, mfu 0.44%
step 2000: train loss 4.0651, val loss 4.0731
iter 2000: loss 1.7938, time 1402.30ms, mfu 0.47%
iter 2500: loss 1.4568, time 282.50ms, mfu 0.78%
step 3000: train loss 4.6135, val loss 4.6220
iter 3000: loss 1.1881, time 1403.46ms, mfu 0.78%
iter 3500: loss 0.8719, time 282.30ms, mfu 1.06%
step 4000: train loss 5.2243, val loss 5.2386
iter 4000: loss 0.6557, time 1404.45ms, mfu 1.02%
iter 4500: loss 0.4472, time 282.40ms, mfu 1.28%
step 5000: train loss 5.8124, val loss 5.8276
iter 5000: loss 0.3386, time 1402.32ms, mfu 1.22%
iter 5500: loss 0.2327, time 283.44ms, mfu 1.46%
step 6000: train loss 6.3483, val loss 6.3677
iter 6000: loss 0.1596, time 1401.30ms, mfu 1.39%
iter 6500: loss 0.1236, time 283.24ms, mfu 1.61%
step 7000: train loss 6.8042, val loss 6.8

KeyboardInterrupt: 

In [ ]:
sample_cursor

In [ ]:
# generate from the model

context = torch.tensor(encode("what is happening here _"), dtype = torch.long , device = device_type).view(1,-1)
print(context)
print(decode(raw_model.generate(context, max_new_tokens=15)[0].tolist()))
